# IGRINS Data Reduction: Raw Data to PCA

## Full tutorial based on the IGRINS_transit code found at https://github.com/meganmansfield/IGRINS_transit
### This code is currently under heavy development, but the tutorial included here, which can also be found on the GitHub page, will take you from raw data to a principal component analysis. Future updates will include cross-correlation analysis. This tutorial follows the process outlined in run_pipeline.py
### NOTE: This tutorial is based on as-yet-unpublished data. Please do not share the data/results without permission.
### First, let's load the analysis packages

In [ ]:
import numpy as np
import make_cube #makes the data cube and calculates barycentric velocity
import wavecal #performs wavelength calibration
import do_pca #performs principal component analysis

### Now feed the code all the important parameters.

In [ ]:
path='/Users/megan/Documents/Projects/GeminiTransitSurvey/WASP76/20211029/reduced/' #path to the reduced data provided by the IGRINS pipeline package (PLP)
date='20211029' #date of the observations (just used to name folders)
Tprimary_UT='2021-10-30T04:20:00.000' #time of primary transit, which can be found using the Transit and Ephemeris Service tool at https://exoplanetarchive.ipac.caltech.edu/
Per=1.809886 #in days, also from Exoplanet Archive or your favorite planet lookup service
radeg=26.6329167 #in degrees
decdeg=2.7003889 #in degrees
#note that RA and Dec must be in degrees; here's a helpful converter: http://www.astrouw.edu.pl/~jskowron/ra-dec/
skyorder=1 #set to 1 if the sky frame was observed at the start of the night and 2 if the sky frame was observed at the end of the night
Vsys=-1.109 #Systemic radial velocity in km/s, listed as "gamma" on the Exoplanet Archive

### Now we'll set a couple parameters for trimming the data. I usually start with keeping as much data as possible and looking at the SNR before throwing any out.

In [ ]:
badorders=np.array([]) #set which orders will get ignored. Lower numbers are at the red end of the spectrum, and IGRINS has 54 orders
trimedges=np.array([100,-100]) #set how many points will get trimmed off the edges of each order: fist number is blue edge, second number is red edge

### Now we're ready to make our data cube! This function will take in the raw data files and output a cube of data organized by order x file (phase of orbit) x pixels. It also outputs the observed phases and barycentric velocity at each exposure time.

In [ ]:
phi,Vbary,grid_RAW,data_RAW,wlgrid,data=make_cube.make_cube(path,date,Tprimary_UT,Per,radeg,decdeg,skyorder,badorders,trimedges,plot=True,output=False,testorders=True)
#testorders=True will output some information on the SNR of all the orders.
#plot=True will plot the SNR over each order
#output=True will save 3 pickle files for future data processing (not necessary if you stay within this pipeline, but useful if you want to port information to your own data analysis process):
#phi.pic has observed phases
#Vbary.pic has barycentric velocities
#data_raw_OBSDATE.pic has [wavelength grid, raw data (no orders removed), skyorder parameter]

### Now we have to perform a wavelength calibration. The function here just assumes the image taken closest to the sky calibration has "true" wavelengths and does a simple shift/stretch to fit each subsequent spectrum to the template spectrum.

In [ ]:
wlgrid,wavecorrect=wavecal.correct(wlgrid,data,skyorder,plot=True,output=False)
#plot=True will plot a zoom-in on one of the orders so you can see the results of the wavelength calibration and check them by eye
#output=True will save a file called wavelengthcalibrated.pic that contains [wavelength array, wavelength calibrated data cube]

### Now we can perform a principal component analysis! 

In [ ]:
nPCAs=3 #number of principal components to subtract (or maximum value for test_pca)
wlgrid,pca_clean_data,pca_noplanet=do_pca.do_pca(wlgrid,wavecorrect,nPCAs,test_pca=True,plot=True,output=False)
#test_pca=True will go through removing every number of PCs from 1 to nPCAs, plotting the cleaned results each time
#plot=True will show a plot comparing the raw data, data with 1 PC removed, and data with nPCAs removed
#output=True will save two pickle files:
#PCA_n_clean_data.pic contains [wavelength array, cleaned data] where cleaned data have n PCs removed
#PCA_n_noise.pic contains [wavelength array, removed noise] where removed noise is all the signal contained in the first n PCs